<a href="https://colab.research.google.com/github/abhi271998/CPI-and-GI-index-analysis-using-regression-model/blob/main/Emoji_Prediction%20on%20sentences%20using%20glove%206B%2050D%20word%20Embedding%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emoji

     |████████████████████████████████| 133kB 4.3MB/s 


In [2]:
import numpy as np
from scipy import spatial

In [3]:
from google.colab import files
!pip install -q kaggle
uploaded = files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [4]:
!kaggle datasets download -d adityajn105/glove6b50d

 81% 55.0M/67.7M [00:01<00:00, 23.4MB/s]
100% 67.7M/67.7M [00:01<00:00, 38.0MB/s]


In [5]:
!unzip '/content/glove6b50d.zip'

Archive:  /content/glove6b50d.zip
  inflating: glove.6B.50d.txt        


In [9]:
import emoji

In [10]:
def label_to_emoji(label):
  return emoji.emojize(label, use_aliases=True)

In [20]:
label_to_emoji(":smile:")

'😄'

In [12]:
embedding_index = {}

f = open('/content/glove.6B.50d.txt')
for line in f:
  values = line.split()
  word = values[0]
  coef = np.asarray(values[1:], dtype = 'float32')
  embedding_index[word] = coef

f.close()

In [13]:
embedding_index["i"].shape

(50,)

In [14]:
spatial.distance.cosine(embedding_index["happy"], embedding_index["sad"])

0.31093674898147583

In [15]:
spatial.distance.cosine(embedding_index["india"], embedding_index["delhi"])

0.18572336435317993

In [16]:
spatial.distance.cosine(embedding_index["france"], embedding_index["paris"])

0.19746702909469604

In [17]:
spatial.distance.cosine(embedding_index["germany"], embedding_index["berlin"])

0.20145773887634277

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout
import pandas as pd

In [19]:
train = pd.read_csv('/content/train_emoji.csv', header=None)
test = pd.read_csv('/content/test_emoji.csv', header=None)

train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [21]:
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [22]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(132,) (132,)
(56,) (56,)


In [23]:
for ix in range(X_train.shape[0]):
  X_train[ix] = X_train[ix].split()

for ix in range(X_test.shape[0]):
  X_test[ix] = X_test[ix].split()

from tensorflow.keras.utils import to_categorical

Y_train = to_categorical(Y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [24]:
print(X_train[0], Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [25]:
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
  for iy in range(len(X_train[ix])):
    embedding_matrix_train[ix][iy] = embedding_index[X_train[ix][iy].lower()]

for ix in range(X_test.shape[0]):
  for iy in range(len(X_test[ix])):
    embedding_matrix_test[ix][iy] = embedding_index[X_test[ix][iy].lower()]


In [26]:
print(embedding_matrix_train.shape)

(132, 10, 50)


In [27]:
model = Sequential()
model.add(SimpleRNN(64, return_sequences=True, input_shape=(10, 50)))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 10, 64)            7360      
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(embedding_matrix_train, Y_train, epochs=50, batch_size=32, shuffle=True)

Epoch 1/50
5/5 [==============================] - 4s 21ms/step - loss: 1.9265 - accuracy: 0.1705
Epoch 2/50
5/5 [==============================] - 0s 19ms/step - loss: 1.6969 - accuracy: 0.2754
Epoch 3/50
5/5 [==============================] - 0s 20ms/step - loss: 1.7908 - accuracy: 0.2519
Epoch 4/50
5/5 [==============================] - 0s 20ms/step - loss: 1.7778 - accuracy: 0.3067
Epoch 5/50
5/5 [==============================] - 0s 19ms/step - loss: 1.5835 - accuracy: 0.3323
Epoch 6/50
5/5 [==============================] - 0s 21ms/step - loss: 1.5320 - accuracy: 0.3761
Epoch 7/50
5/5 [==============================] - 0s 21ms/step - loss: 1.4246 - accuracy: 0.3882
Epoch 8/50
5/5 [==============================] - 0s 21ms/step - loss: 1.4308 - accuracy: 0.3965
Epoch 9/50
5/5 [==============================] - 0s 20ms/step - loss: 1.2733 - accuracy: 0.5109
Epoch 10/50
5/5 [==============================] - 0s 20ms/step - loss: 1.2142 - accuracy: 0.4746
Epoch 11/50
5/5 [============

In [29]:
pred = model.predict_classes(embedding_matrix_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [30]:
acc = float(sum(pred == Y_test)) / embedding_matrix_test.shape[0]
print(acc) 

0.5357142857142857
